##### <b>Step-1: Importing Libraries</b>

In [1]:
import os                               
import cv2
import time                              
import numpy as np   
import tensorflow as tf 
from PIL import Image
import numpy as np

import tensorflow as tf
from tensorflow import keras
from skimage import transform
from google.colab import drive 
from keras import backend as K           
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from keras.models import load_model
from keras.preprocessing import image  
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D,BatchNormalization,Activation,Conv2DTranspose,MaxPool2D,Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator

##### <b>Step-2: Mounting Drive</b>

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
path="/content/drive/MyDrive/CRACK500-Resized"

In [4]:
os.chdir(path)

In [2]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Conv2D, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load the pre-trained ResNet34 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add upsampling and convolutional layers for segmentation
x = base_model.output
x = UpSampling2D((2, 2))(x)
x = Conv2D(1, (1, 1), activation='sigmoid')(x)  # Output segmentation mask

# Create the segmentation model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy')

# Print model summary
model.summary()


94765736/94765736 [==============================] - 4s 0us/step


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [5]:
os.listdir(path)

['pytorch-road-segmentation-syncrack.ipynb',
 'deeplabv3_plus.ipynb',
 'Test',
 'val',
 'Train',
 'Wound_Segmentation_on_Fuseg_&_Medec_Data.ipynb',
 'pneumonia_detection_using_Unet.ipynb',
 'segmentation.ipynb']

In [5]:
# train_images_path=os.path.join(os.path.join(path,'Train'),'images')
# train_mask_path=os.path.join(os.path.join(path,'Train'),'masks')

In [6]:
# test_images_path=os.path.join(os.path.join(path,'Test'),'images')
# test_mask_path=os.path.join(os.path.join(path,'Test'),'masks')

In [7]:
# val_images_path=os.path.join(os.path.join(path,'val'),'images')
# val_mask_path=os.path.join(os.path.join(path,'val'),'masks')

In [11]:
seed = 909 # (IMPORTANT) to transform image and corresponding mask with same augmentation parameter.
image_datagen = ImageDataGenerator(width_shift_range=0.1,
                 height_shift_range=0.1,) # custom fuction for each image you can use resnet one too.
mask_datagen = ImageDataGenerator(width_shift_range=0.1,
                 height_shift_range=0.1)  # to make mask as feedable formate (256,256,1)

image_generator =image_datagen.flow_from_directory(train_images_path,
                                                    class_mode=None, seed=seed)

mask_generator = mask_datagen.flow_from_directory(train_mask_path,
                                                   class_mode=None, seed=seed)

train_generator = zip(image_generator, mask_generator)

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
seed1 = 901
seed2=902
# Set up ImageDataGenerator for original images
image_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True
)

# Set up ImageDataGenerator for mask images
mask_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True
)

# Load original training images
train_image_generator = image_datagen.flow_from_directory(
    "Train/train_images",
    target_size=(224,224),
    batch_size=32,
    class_mode=None,
    shuffle=True,
    seed=seed1
)

# Load mask training images
train_mask_generator = mask_datagen.flow_from_directory(
    "Train/train_masks",
    target_size=(224,224),
    batch_size=32,
    class_mode=None,
    shuffle=True,
    seed=seed1
)

# Load original validation images
val_image_generator = image_datagen.flow_from_directory(
    "Test/test_images",
    target_size=(224,224),
    batch_size=8,
    class_mode=None,
    shuffle=True,
    seed=seed2
)

# Load mask validation images
val_mask_generator = mask_datagen.flow_from_directory(
    "Test/test_masks",
    target_size=(224,224),
    batch_size=8,
    class_mode=None,
    shuffle=True,
    seed=seed2
)

# Combine the training generators into a single generator
train_generator = zip(train_image_generator, train_mask_generator)

# Combine the validation generators into a single generator
val_generator = zip(val_image_generator, val_mask_generator)




Found 1896 images belonging to 1 classes.
Found 1896 images belonging to 1 classes.
Found 1124 images belonging to 1 classes.
Found 1124 images belonging to 1 classes.


In [ ]:
ffff

In [7]:
import tensorflow as tf

# Define custom IoU metric function
def iou_score(y_true, y_pred):
    y_true = tf.cast(y_true > 0.5, dtype=tf.float32)
    y_pred = tf.cast(y_pred > 0.5, dtype=tf.float32)
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
    iou = intersection / union
    return iou
def jaccard_distance(y_true, y_pred, smooth=100):
    """ Calculates mean of Jaccard distance as a loss function """
    intersection = tf.reduce_sum(y_true * y_pred, axis=(1,2))
    sum_ = tf.reduce_sum(y_true + y_pred, axis=(1,2))
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    jd =  (1 - jac) * smooth
    return tf.reduce_mean(jd)

In [ ]:
# import tensorflow as tf
# import tensorflow.keras.layers as L
# from tensorflow.keras.models import Model

# # Define precision and recall as metrics
# precision = tf.keras.metrics.Precision()
# recall = tf.keras.metrics.Recall()

# def conv_block(x, num_filters):
#     x = L.Conv2D(num_filters, 3, padding="same")(x)
#     x = L.BatchNormalization()(x)
#     x = L.Activation("relu")(x)

#     x = L.Conv2D(num_filters, 3, padding="same")(x)
#     x = L.BatchNormalization()(x)
#     x = L.Activation("relu")(x)

#     return x

# def encoder_block(x, num_filters):
#     x = conv_block(x, num_filters)
#     p = L.MaxPool2D((2, 2))(x)
#     return x, p

# def attention_gate(g, s, num_filters):
#     Wg = L.Conv2D(num_filters, 1, padding="same")(g)
#     Wg = L.BatchNormalization()(Wg)

#     Ws = L.Conv2D(num_filters, 1, padding="same")(s)
#     Ws = L.BatchNormalization()(Ws)

#     out = L.Activation("relu")(Wg + Ws)
#     out = L.Conv2D(num_filters, 1, padding="same")(out)
#     out = L.Activation("sigmoid")(out)

#     return out * s

# def decoder_block(x, s, num_filters):
#     x = L.UpSampling2D(interpolation="bilinear")(x)
#     s = attention_gate(x, s, num_filters)
#     x = L.Concatenate()([x, s])
#     x = conv_block(x, num_filters)
#     return x

# def attention_unet(input_shape):
#     """ Inputs """
#     inputs = L.Input(input_shape)

#     """ Encoder """
#     s1, p1 = encoder_block(inputs, 64)
#     s2, p2 = encoder_block(p1, 128)
#     s3, p3 = encoder_block(p2, 256)

#     b1 = conv_block(p3, 512)

#     """ Decoder """
#     d1 = decoder_block(b1, s3, 256)
#     d2 = decoder_block(d1, s2, 128)
#     d3 = decoder_block(d2, s1, 64)

#     """ Outputs """
#     outputs = L.Conv2D(1, 1, padding="same", activation="sigmoid")(d3)

#     """ Model """
#     model = Model(inputs, outputs, name="Attention-UNET")
#     return model
# input_shape = (224,224, 3)
# model = attention_unet(input_shape)

# model.compile(optimizer="adam", loss=jaccard_distance, metrics=[precision,recall,iou_score])
# model.summary()

In [8]:
!pip3 install -U segmentation-models
%env SM_FRAMEWORK=tf.keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 5.8 MB/s eta 0:00:00
env: SM_FRAMEWORK=tf.keras


In [10]:
import segmentation_models as sm
# Compile the model
import tensorflow as tf

# Define precision and recall as metrics
precision = tf.keras.metrics.Precision()
recall = tf.keras.metrics.Recall()

# binary segmentation (this parameters are default when you call Unet('resnet34')
model = sm.Unet('resnet101',input_shape=(224,224,3),encoder_weights='imagenet', classes=1, activation='sigmoid')
preprocess_input = sm.get_preprocessing('resnet101')
# train_images = preprocess_input(train_images)
# val_images = preprocess_input(val_images)
model.compile(
    'Adam',
    loss=sm.losses.bce_jaccard_loss,
    metrics=[sm.metrics.iou_score,precision,recall],
)

model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 data (InputLayer)              [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 bn_data (BatchNormalization)   (None, 224, 224, 3)  9           ['data[0][0]']                   
                                                                                                  
 zero_padding2d_35 (ZeroPadding  (None, 230, 230, 3)  0          ['bn_data[0][0]']                
 2D)                                                                                              
                                                                                            

# Model Compilation

In [ ]:
# # # Define precision and recall as metrics
# import tensorflow as tf
# precision = tf.keras.metrics.Precision()
# recall = tf.keras.metrics.Recall()
# model = UNet()
# model.compile(optimizer="adam", loss=jaccard_distance, metrics=[precision,recall,iou_score])
# model.summary()

In [13]:
# Train the segmentation model using the training generator
hist=model.fit_generator(
    train_generator,
    epochs=50,
    validation_data=val_generator,
)





<ipython-input-13-9362d6717c5a>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist=model.fit_generator(


Epoch 1/50


InvalidArgumentError: ignored

In [ ]:
history = model.fit(train_images,train_masks,
                          batch_size=2,epochs=40,verbose = 1,validation_data=(val_images,val_masks)
                          )

In [ ]:
import matplotlib.pyplot as plt
import math

# Predict the masks for validation images
pred_masks = model.predict(val_images)

# Define the number of rows and columns for displaying the images
num_images = 3

# Select random indices for displaying the images
indices = np.random.choice(len(val_images), size=num_images, replace=False)

# Create subplots for displaying the images
fig, axes = plt.subplots(num_images, 3, figsize=(12, 12))

# Iterate over the selected indices and display the images
for i, index in enumerate(indices):
    # Display the original image
    axes[i, 0].imshow(val_images[index])
    axes[i, 0].set_title('Original Image')
    axes[i, 0].axis('off')
    
    # Display the predicted mask
    axes[i, 1].imshow(pred_masks[index], cmap='gray')
    axes[i, 1].set_title('Predicted Mask')
    axes[i, 1].axis('off')
    
    # Display the validation mask
    axes[i, 2].imshow(val_masks[index], cmap='gray')
    axes[i, 2].set_title('Validation Mask')
    axes[i, 2].axis('off')

# Adjust the spacing between subplots
plt.tight_layout()

# Show the figure
plt.show()


In [ ]:
fig = plt.figure()
fig.subplots_adjust(hspace=0.4, wspace=0.4)

ax = fig.add_subplot(1, 2, 1)
ax.imshow(np.reshape(masks[0]*255, (image_size, image_size)), cmap="gray")

ax = fig.add_subplot(1, 2, 2)
ax.imshow(np.reshape(result[0]*255, (image_size, image_size)), cmap="gray")